# MOQC: a comprehensive tutorial

In [ ]:
%cd /data/marciano/experiments/multi-organ-qc/moqc

In [ ]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
from utils.common import Visualizer
from utils.dataset import select_valid_imgs, remove_non_common_files

In [ ]:
rand_img = nib.load(f'/data/marciano/experiments/multi-organ-qc/MSD_data/Task01_BrainTumour/labelsTr/BRATS_001.nii.gz').get_fdata().transpose(2, 0, 1)
rand_img.shape

In [ ]:
viz = Visualizer(rand_img=rand_img)
viz.plot_3d()

In [ ]:
label_path = '/data/marciano/experiments/multi-organ-qc/MSD_data/Task01_BrainTumour/labelsTr/'
labsave = '/data/marciano/experiments/multi-organ-qc/data/brain/labels'
select_valid_imgs(label_path, labsave, inter_slice_range=[50, 120], non_zero_thres=0.005)

segpath = '/data/marciano/experiments/multi-organ-qc/nnUnet_seg/brain/Tr/'
segsave = '/data/marciano/experiments/multi-organ-qc/data/brain/nnunet/segmentations'
select_valid_imgs(segpath, segsave, inter_slice_range=[50, 120], non_zero_thres=0.005)

In [ ]:
remove_non_common_files(labsave, segsave)

In [ ]:
assert len(os.listdir(labsave)) == len(os.listdir(segsave))
assert os.listdir(labsave).sort() == os.listdir(segsave).sort()

In [ ]:
%cd /data/marciano/experiments/multi-organ-qc

In [ ]:
!python moqc/data_preparation.py -og prostate -k ['prostate']
!python moqc/data_preparation.py -d data/prostate/nnunet/ -mf -segmentations -og '' --k ['prostate']

In [ ]:
!python moqc/train.py -og prostate --model small_cae

In [ ]:
!python moqc/test.py -og prostate -m small_cae

In [ ]:
!python moqc/evaluate.py -og prostate -m small_cae -seg nnunet -l -c